# Multiprocessing: processes; fork & wait
_COSC 208, Introduction to Computer Systems, 2023-04-14_

## Announcements
* Project 4 due Thursday, April 20

## Outline
* Process abstraction
* Creating processes
* Waiting for processes

## No warm-up — Happy Friday!

## Process abstraction

* Process — running program and its corresponding machine state (code, memory, and register values)
    * Program is static code and static data; process is dynamic instance of code and data
    * Cooking analogy
        * Recipe == program
        * Following a recipe == process
        * Ingredients == program inputs
        * Prepared food == program outputs
        * Cabinets, fridge, etc. == persistent storage
        * Prep area (e.g., counter) == registers & main memory
        * Contents and status of the prep area; current step of recipe == machine state
        * Chef == CPU
    * Can have multiple processes all running different instances of the same program
        * Cooking analogy — chef may be making multiple batches of the same recipe
* OS is responsible for...
    * Creating processes — when a user or another application requests it
    * Scheduling processes — i.e., deciding when/which process should be allowed to use the CPU
    * Switching processes — i.e., saving the machine state of one process and restoring the machine state of another process; called context switching
    * Cleaning-up processes — i.e., releasing any resources they are using when the process is done
    * Interacting with I/O devices (e.g., disks, NICs, graphics card) on behalf of processes

* Q1: _Consider building a Lego kit as an analogy for operating systems' process abstraction. Match each component of the analogy with the corresponding component of a real computer system._

* Analogy
    1. Cabinet/drawers for storing Legos
    2. Lego bricks
    3. Building area (e.g., tabletop)
    4. Instruction booklet
    5. Following the assembly instructions
    6. Current step for the instruction booklet
    7. Completed kit
    8. You
* Real system
    * CPU
    * persistent storage
    * process
    * program
    * program counter
    * program inputs
    * program outputs
    * registers and main memory

* Instruction booklet == program
* Following the assembly instructions == process
* Lego bricks == program inputs
* Completed kit == program outputs
* Cabinet/drawers for storing Legos == persistent storage
* Building area (e.g., tabletop) == registers and main memory
* Current step for the instruction booklet == program counter
* You == CPU

🛑 **STOP HERE** after completing the above question; if you have extra time please **skip ahead** to the extra practice.

## Creating processes

* `int fork()`
    * Creates an exact copy of the running process, except for the return value from `fork` — return `0` to child (i.e., new) process; return child’s process ID to parent process (i.e., process that called fork)
    * Both child and parent resume execution from place where `fork` was called

* Q2: _What does the following code output?_

In [3]:
#include <stdio.h>
#include <unistd.h>
int main(int argc, char **argv) {
    printf("Before fork\n");
    fork();
    printf("After fork\n");
    return 0;
}

Before fork
After fork
After fork


<p style="height:10em;"></p>

* Q3: _What does the following code output (assuming the new process has PID 1819)?_

In [ ]:
#include <stdio.h>
#include <unistd.h>    
int main(int argc, char **argv) {
    printf("Before fork\n");
    int pid = fork();
    if (pid < 0) {
        printf("Fork failed\n");
        return 1;
    } else if (pid == 0) {
        printf("Child gets %d\n", pid);
    } else {
        printf("Parent gets %d\n", pid);
    }
    return 0;
}

```
Before fork
Child gets 0
Parent gets 1819
```
OR
```
Before fork
Parent gets 1819
Child gets 0
``` 

<p style="height:10em;"></p>

🛑 **STOP HERE** after completing the above questions; if you have extra time please **skip ahead** to the extra practice.

## Waiting for processes

* Wait for any child to finish — `int wait(int *status)`
    * Returns PID of the child process that finished
    * `status` parameter is optional
        * if passed a valid integer pointer, `wait` will store the return value of the child process's `main` function at the referenced memory location
        * if return value of child process's main function is not needed, then pass `NULL`
* Wait for a specific process to finish — `int waitpid(pid_t pid, int *status, int options)`
    * Returns PID of the process that finished
    * `pid` is PID if process to wait for — need not be a child process
    * `status` is the same as `wait`
    * `options` is typically `0`, except in special circumstances
* Wait functions do not return until child or specific process, respectively, finishes

* Q4: _What are all possible outputs of this program?_

In [19]:
#include <stdio.h>
#include <unistd.h>
int main() {
    int pid = fork();
    if (pid < 0) {
        printf("Fork failed\n");
        return 1;
    } else if (pid == 0) {
        printf("Child\n");
    } else {
        wait(NULL);
        printf("Parent\n");
    }
    return 0;
}

Child
Parent


<p style="height:12em;"></p>

* Q5: _What are all possible outputs of this program (assuming the new process has PID 13346)?_

In [ ]:
#include <stdio.h>
#include <unistd.h>
int main() {
    int pid = fork();
    printf("A %d\n", pid);
    if (pid < 0) {
        printf("Fork failed\n");
        return 1;
    } else if (pid == 0) {
        printf("B\n");
    } else {
        wait(NULL);
        printf("C\n");
    }
}

```
A 0
A 13346
B
C
```
OR
```
A 13346
A 0
B
C
```
OR
```
A 0
B
A 13346
C
```

<p style="height:10em;"></p>

## Extra practice

* Q6: _What does the following code output?_

In [ ]:
#include <stdio.h>
#include <unistd.h>
int main(int argc, char **argv) {
    int value = 100;
    int pid = fork();
    if (pid < 0) {
        printf("Fork failed\n");
        return 1;
    } else if (pid == 0) {
        value -= 50;
    } else {
        value += 50;
    }
    printf("My value is %d\n", value);
    return 0;
}

```
My value is 50
My value is 150
```
OR
```
My value is 150
My value is 50
```

<p style="height:12em;"></p>

* Q7: _What does the following code output?_

In [ ]:
#include <stdio.h>
#include <unistd.h>
int main(int argc, char **argv) {
    printf("Begin\n");
    int pid = fork();
    if (pid < 0) {
        printf("Fork failed\n");
        return 1;
    } else if (pid == 0) {
        printf("Child\n");
        return 0;
    } else {
        printf("Parent\n");
    }
    printf("End\n");
}

```
Begin
Child
Parent
End 
```
OR
```
Begin
Parent
Child
End 
```
OR
```
Begin
Parent
End 
Child
```

<p style="height:15em;"></p>

* Q8: _How would you modify the above program such that `Child` always prints before `Parent`?_

In [17]:
#include <stdio.h>
#include <unistd.h>
int main(int argc, char **argv) {
    printf("Begin\n");
    int pid = fork();
    if (pid < 0) {
        printf("Fork failed\n");
        return 1;
    } else if (pid == 0) {
        printf("Child\n");
        return 0;
    } else {
        wait(NULL);
        printf("Parent\n");
    }
    printf("End\n");
}

Begin
Child
Parent
End
